In [3]:
from __future__ import print_function
import cobra
import os
import libsbml
import pandas as pd

In [4]:
strains = ["HM1", "HM3", "HM6", "HM7", "HM14", "HM17", "HM43", "HM54", "HM56","HM57", "HM66", "HM68", "HM86"]
fnames = []
for st in strains:
    PATH="/Users/annasintsova/git_repos/HUTI-RNAseq/analysis/fba/2017-12-16-model-2/data/{}_homologues/{}_model/{}_iJO1366.xml".format(st,st,st)
    fnames.append(PATH)

In [5]:
#Load the models
hm01, hm03, hm06, hm07, hm14, hm17, hm43, hm54, hm56, hm57, hm66, hm68, hm86 = map(cobra.io.read_sbml_model, fnames)
models = [hm01, hm03, hm06, hm07, hm14, hm17, hm43, hm54, hm56, hm57, hm66, hm68, hm86]

In [6]:
for mod in models:
    print(mod)
    print(mod.optimize().objective_value)

HM1_iJO1366
0.4024743606721175
HM3_iJO1366
0.9823568206923815
HM6_iJO1366
0.9823718127269756
HM7_iJO1366
0.8294597663804111
HM14_iJO1366
0.9792996024228706
HM17_iJO1366
0.9706718035894507
HM43_iJO1366
0.40247730149072225
HM54_iJO1366
0.9755115436722549
HM56_iJO1366
0.982371812726981
HM57_iJO1366
0.9823568206923835
HM66_iJO1366
0.8560604811010539
HM68_iJO1366
0.9823718127269833
HM86_iJO1366
0.9662463433080954


In [104]:
carbon_sources = ['acetate exchange', 'succinate exchange', 'acetaldehyde exchange', '2-oxoglutarate exchange',
                 'ethanol exchange', 'fumarate exchange', 'd-fructose exchange', 'd-glucose exchange', 'l-glutamine exchange', 
                 'l-glutamate exchange', 'd-lactate exchange', 'l-malate exchange', 'pyruvate exchange']
colnames = []
aerobic_results = []
anaerobic_results = []

for mod in models:
    colnames.append(mod)
    exchange_subs = {}
    for rex in mod.reactions:
        if "EX_" in rex.id:
            exchange_subs[rex.name]=rex.id
    glc = mod.reactions.EX_glc_LPAREN_e_RPAREN_
    glc.lower_bound = 0.0
    ox = mod.reactions.EX_o2_LPAREN_e_RPAREN_
    aerobic_biomass = {}
    anaerobic_biomass ={}
    for key, val in exchange_subs.items():
        if key.lower() in carbon_sources:
            cs = mod.reactions.get_by_id(val)
            cs.lower_bound = -10
            aerobic_biomass[key] = round(mod.optimize().objective_value,4)
            ox.lower_bound = 0.0
            anaerobic_biomass[key] = round(mod.optimize().objective_value,4)
            cs.lower_bound = 0
            ox.lower_bound = -1000.0
    glc.lower_bound = -10.0
    aerobic_results.append(aerobic_biomass)
    anaerobic_results.append(anaerobic_biomass)
aerobic = pd.DataFrame(aerobic_results).T
aerobic.columns = colnames
anaerobic = pd.DataFrame(anaerobic_results).T
anaerobic.columns = colnames
anaerobic.to_csv("/Users/annasintsova/git_repos/HUTI-RNAseq/analysis/fba/2017-12-16-model-2/analysis/2018-01-05-modeling-growth-rate-anaerobic.csv")
aerobic.to_csv("/Users/annasintsova/git_repos/HUTI-RNAseq/analysis/fba/2017-12-16-model-2/analysis/2018-01-05-modeling-growth-rate-aerobic.csv")

cobra/util/solver.py:419 UserWarning: solver status is 'infeasible'


In [29]:
iron_influx = [-0.1, -0.01, -0.001, -0.0001, -0.00001, 0]
colnames = []
iron_air = []
for mod in models:
    colnames.append(mod)
    mod.reactions.EX_fe3_LPAREN_e_RPAREN_.lower_bound = 0
    iron = mod.reactions.EX_fe2_LPAREN_e_RPAREN_
    ox = mod.reactions.EX_o2_LPAREN_e_RPAREN_
    iron_air_biomass = {}   
    for influx in iron_influx:        
        iron.lower_bound = influx
        iron_air_biomass[influx] = round(mod.optimize().objective_value,4)       
    iron.lower_bound = -1000.0
    iron_air.append(iron_air_biomass)
iron_air_df = pd.DataFrame(iron_air).T
iron_air_df.columns = colnames
iron_air_df
iron_air_df.to_csv("/Users/annasintsova/git_repos/HUTI-RNAseq/analysis/fba/2017-12-16-model-2/analysis/2018-01-05-modeling-iron-limitation-aerobic.csv")
#aerobic.to_csv("/Users/annasintsova/git_repos/HUTI-RNAseq/analysis/fba/2017-12-16-model-2/analysis/2018-01-05-modeling-growth-rate-aerobic.csv")

cobra/util/solver.py:419 UserWarning: solver status is 'infeasible'


In [24]:
ox = hm01.reactions.EX_o2_LPAREN_e_RPAREN_
iron = hm01.reactions.EX_fe3_LPAREN_e_RPAREN_
iron.lower_bound = -1000
ox.lower_bound = 0
hm01.optimize().objective_value

0.16253147110057473

In [38]:
iron_influx = [-1000, -100, -10, -1, -0.1, 0]
colnames = []
iron_no_air = []

for mod in models:
    colnames.append(mod)
    mod.reactions.EX_fe3_LPAREN_e_RPAREN_.lower_bound = -1000
    iron = mod.reactions.EX_fe2_LPAREN_e_RPAREN_
    ox = mod.reactions.EX_o2_LPAREN_e_RPAREN_
    ox.lower_bound = 0.0
    iron_no_air_biomass ={}
    
    for influx in iron_influx:        
        iron.lower_bound = influx   
        iron_no_air_biomass[influx] = round(mod.optimize().objective_value,4)
    ox.lower_bound = -1000.0    
    iron.lower_bound = -1000.0
    iron_no_air.append(iron_no_air_biomass)
iron_no_air_df = pd.DataFrame(iron_no_air).T
iron_no_air_df.columns = colnames
iron_no_air_df
iron_no_air_df.to_csv("/Users/annasintsova/git_repos/HUTI-RNAseq/analysis/fba/2017-12-16-model-2/analysis/2018-01-05-modeling-iron-limitation-anaerobic.csv")

cobra/util/solver.py:419 UserWarning: solver status is 'infeasible'
